### PPOCR 형식의 STD 데이터 셋을 PPOCR 형식 STR 데이터 셋으로 변환
1. 이미지에서 텍스트만 잘라서 저장
2. 레이블 파일 생성

In [1]:
from dataset import *
from tools import polygon_utility

def make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name = "label.txt"):
    assert not save_dir.exists(), f"please remove {save_dir}"
    label_path = save_dir/label_file_name
    image_idx = 0

    for i in range(len(dataset)):  
        try:
            image = dataset.get_x(i)["image"]
            labels = dataset.get_y(i)
                
            for label in labels:
                polygon =  label["polygon"]
                text = label["text"]
                
                cropped_image = polygon_utility.crop_by_polygon(image, polygon)
                image_path = save_dir/f"{(image_idx//dir_size + 1)}"/f"{image_idx+1}.png"
                image_path.parent.mkdir(parents=True, exist_ok=True)
                cropped_image.save(image_path)
                with open(label_path, "a") as f:
                    image = str(image_path.relative_to(save_dir)).replace('\\', '/')
                    f.write(f"{image}\t{text}\n")

                image_idx += 1
        except:
            continue

In [3]:
order = [["sangmu_std_MH", "sangmu_str_MH"],
        ["sangmu_std_DI", "sangmu_str_DI"],
        ["sangmu_std_DH", "sangmu_str_DH"]]

for _std, _str in order:
    dataset = PPOCR_STD_Dataset_Loader(f"E:/workspace/paddleocr/datasets/{_std}", label_file_name="label.txt")
    save_dir = Path(f"E:/workspace/paddleocr/datasets/{_str}")
    make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name="label.txt")

